In [7]:
# Cell 1: Setup and Data Loading
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
import lightgbm as lgb
import optuna
import joblib
import warnings
warnings.filterwarnings('ignore')

# Paths
TRAIN_PATH = "../data/train_processed_comprehensive.csv"
TEST_PATH = "../data/test_processed_comprehensive.csv"
MODEL_PATH = "../models/academic_risk_model.joblib"
os.makedirs("../models", exist_ok=True)

print("🚀 MULTICLASS CLASSIFICATION SETUP")
print("=" * 50)
print("✅ Libraries imported (including Optuna)")
print("✅ Paths configured")
print("✅ Model directory created")
print("🎯 Target: Multiclass Classification (Grade Categories)")


🚀 MULTICLASS CLASSIFICATION SETUP
✅ Libraries imported (including Optuna)
✅ Paths configured
✅ Model directory created
🎯 Target: Multiclass Classification (Grade Categories)


In [8]:
# Cell 2: Load Data and Convert to Multiclass Classification
print("📊 LOADING DATA AND CONVERTING TO MULTICLASS CLASSIFICATION")
print("=" * 60)

# Load the comprehensive processed datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print(f"✅ Training data: {train_df.shape[0]:,} samples, {train_df.shape[1]} features")
print(f"✅ Test data: {test_df.shape[0]:,} samples, {test_df.shape[1]} features")

# Convert GPA to Grade Categories for Multiclass Classification
def gpa_to_grade_category(gpa):
    """Convert GPA to grade categories for multiclass classification"""
    if gpa >= 3.7:
        return 'A'  # A, A-
    elif gpa >= 3.0:
        return 'B'  # B+, B, B-
    elif gpa >= 2.0:
        return 'C'  # C+, C, C-
    elif gpa >= 1.0:
        return 'D'  # D+, D
    else:
        return 'F'  # F

# Apply grade category conversion
train_df['grade_category'] = train_df['gpa'].apply(gpa_to_grade_category)
test_df['grade_category'] = test_df['gpa'].apply(gpa_to_grade_category)

# Prepare features and target
id_cols = ["student_id", "course_id"]
target_col = "grade_category"
feature_cols = [c for c in train_df.columns if c not in id_cols + [target_col, 'gpa']]

# Ensure we only have numerical features
print(f"🔍 Checking feature types...")
print(f"   Total columns: {len(train_df.columns)}")
print(f"   Feature columns: {len(feature_cols)}")
print(f"   ID columns: {len(id_cols)}")
print(f"   Target column: {target_col}")

# Check for any non-numerical features
non_numerical = train_df[feature_cols].select_dtypes(exclude=[np.number]).columns.tolist()
if non_numerical:
    print(f"⚠️ Found non-numerical features: {non_numerical}")
    print("   Removing non-numerical features...")
    feature_cols = [c for c in feature_cols if c not in non_numerical]
    print(f"   Updated feature columns: {len(feature_cols)}")

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

print(f"\n✅ Final feature matrix shape: {X_train.shape}")
print(f"✅ Target vector shape: {y_train.shape}")
print(f"✅ Target classes: {sorted(y_train.unique())}")

# Store original target values for display
y_train_original = y_train.copy()
y_test_original = y_test.copy()

# Encode target labels to numerical values for some algorithms
print(f"\n🔧 Encoding target labels...")
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print(f"   Original classes: {sorted(y_train.unique())}")
print(f"   Encoded classes: {sorted(np.unique(y_train_encoded))}")
print(f"   Class mapping: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

# Use encoded targets for training
y_train = y_train_encoded
y_test = y_test_encoded

print(f"\n📊 Dataset Overview:")
print(f"   Features: {len(feature_cols)}")
print(f"   Target: Grade Categories (A, B, C, D, F)")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")

print(f"\n📊 Target Distribution (Training):")
grade_dist = pd.Series(y_train_original).value_counts()
for grade, count in grade_dist.items():
    print(f"   Grade {grade}: {count:,} samples ({count/len(y_train_original)*100:.1f}%)")

print(f"\n📊 Target Distribution (Test):")
test_grade_dist = pd.Series(y_test_original).value_counts()
for grade, count in test_grade_dist.items():
    print(f"   Grade {grade}: {count:,} samples ({count/len(y_test_original)*100:.1f}%)")

# Check for any missing values
if X_train.isna().sum().sum() > 0 or np.isnan(y_train).sum() > 0:
    print(f"\n⚠️ Missing values detected!")
    print(f"   X_train missing: {X_train.isna().sum().sum()}")
    print(f"   y_train missing: {np.isnan(y_train).sum()}")
else:
    print(f"\n✅ No missing values in training data")

# Check class balance
print(f"\n📊 Class Balance Analysis:")
print(f"   Number of classes: {len(grade_dist)}")
print(f"   Most frequent class: {grade_dist.idxmax()} ({grade_dist.max()} samples)")
print(f"   Least frequent class: {grade_dist.idxmin()} ({grade_dist.min()} samples)")
print(f"   Class imbalance ratio: {grade_dist.max() / grade_dist.min():.2f}:1")


📊 LOADING DATA AND CONVERTING TO MULTICLASS CLASSIFICATION
✅ Training data: 3,281 samples, 143 features
✅ Test data: 821 samples, 143 features
🔍 Checking feature types...
   Total columns: 144
   Feature columns: 140
   ID columns: 2
   Target column: grade_category
⚠️ Found non-numerical features: ['grade']
   Removing non-numerical features...
   Updated feature columns: 139

✅ Final feature matrix shape: (3281, 139)
✅ Target vector shape: (3281,)
✅ Target classes: ['A', 'B', 'C', 'D', 'F']

🔧 Encoding target labels...
   Original classes: ['A', 'B', 'C', 'D', 'F']
   Encoded classes: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
   Class mapping: {'A': np.int64(0), 'B': np.int64(1), 'C': np.int64(2), 'D': np.int64(3), 'F': np.int64(4)}

📊 Dataset Overview:
   Features: 139
   Target: Grade Categories (A, B, C, D, F)
   Training samples: 3,281
   Test samples: 821

📊 Target Distribution (Training):
   Grade A: 1,011 samples (30.8%)
   Grade B: 987 samples (30.1%)


In [9]:
# Cell 3: Baseline Classification Models
print("🎯 TRAINING BASELINE CLASSIFICATION MODELS")
print("=" * 50)

# 1. Logistic Regression (Baseline)
print("📊 Training Logistic Regression...")
lr_model = LogisticRegression(
    multi_class='ovr',  # One-vs-Rest for multiclass
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr, average='weighted')
lr_precision = precision_score(y_test, y_pred_lr, average='weighted')
lr_recall = recall_score(y_test, y_pred_lr, average='weighted')

print(f"✅ Logistic Regression Results:")
print(f"   Accuracy: {lr_accuracy:.4f}")
print(f"   F1-Score: {lr_f1:.4f}")
print(f"   Precision: {lr_precision:.4f}")
print(f"   Recall: {lr_recall:.4f}")

# 2. Decision Tree
print("\n📊 Training Decision Tree...")
dt_model = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt, average='weighted')
dt_precision = precision_score(y_test, y_pred_dt, average='weighted')
dt_recall = recall_score(y_test, y_pred_dt, average='weighted')

print(f"✅ Decision Tree Results:")
print(f"   Accuracy: {dt_accuracy:.4f}")
print(f"   F1-Score: {dt_f1:.4f}")
print(f"   Precision: {dt_precision:.4f}")
print(f"   Recall: {dt_recall:.4f}")

# 3. Random Forest
print("\n📊 Training Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')
rf_precision = precision_score(y_test, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test, y_pred_rf, average='weighted')

print(f"✅ Random Forest Results:")
print(f"   Accuracy: {rf_accuracy:.4f}")
print(f"   F1-Score: {rf_f1:.4f}")
print(f"   Precision: {rf_precision:.4f}")
print(f"   Recall: {rf_recall:.4f}")

# Compare baselines
print(f"\n📊 Baseline Model Comparison:")
print(f"   Logistic Regression F1: {lr_f1:.4f}")
print(f"   Decision Tree F1: {dt_f1:.4f}")
print(f"   Random Forest F1: {rf_f1:.4f}")

best_baseline = max([(lr_f1, "Logistic Regression"), (dt_f1, "Decision Tree"), (rf_f1, "Random Forest")])
print(f"\n🏆 Best Baseline: {best_baseline[1]} (F1 = {best_baseline[0]:.4f})")


🎯 TRAINING BASELINE CLASSIFICATION MODELS
📊 Training Logistic Regression...
✅ Logistic Regression Results:
   Accuracy: 0.3386
   F1-Score: 0.3246
   Precision: 0.3262
   Recall: 0.3386

📊 Training Decision Tree...
✅ Decision Tree Results:
   Accuracy: 0.3045
   F1-Score: 0.2956
   Precision: 0.2924
   Recall: 0.3045

📊 Training Random Forest...
✅ Random Forest Results:
   Accuracy: 0.3520
   F1-Score: 0.3275
   Precision: 0.3183
   Recall: 0.3520

📊 Baseline Model Comparison:
   Logistic Regression F1: 0.3246
   Decision Tree F1: 0.2956
   Random Forest F1: 0.3275

🏆 Best Baseline: Random Forest (F1 = 0.3275)


In [10]:
# Cell 4: LightGBM with Optuna Hyperparameter Tuning
print("🚀 LIGHTGBM WITH OPTUNA HYPERPARAMETER TUNING")
print("=" * 60)

# Define objective function for Optuna
def objective(trial):
    """Objective function for Optuna optimization"""
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': len(np.unique(y_train)),
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }
    
    # Use cross-validation for robust evaluation
    model = lgb.LGBMClassifier(**params)
    
    # 3-fold stratified CV
    cv_scores = cross_val_score(
        model, X_train, y_train, 
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
        scoring='f1_weighted',
        n_jobs=1  # Avoid nested parallelism
    )
    
    return cv_scores.mean()

# Create Optuna study
print("🔍 Creating Optuna study...")
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)

# Run optimization
n_trials = 50  # Good balance for this dataset size
print(f"🚀 Running {n_trials} optimization trials...")
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

print(f"\n✅ Optimization Complete!")
print(f"   Best F1 Score: {study.best_value:.4f}")
print(f"   Best Parameters:")
for key, value in study.best_params.items():
    print(f"     {key}: {value}")

# Train final model with best parameters
print(f"\n📊 Training Final LightGBM with Best Parameters...")
best_lgb_model = lgb.LGBMClassifier(**study.best_params, random_state=42, verbose=-1)
best_lgb_model.fit(X_train, y_train)
y_pred_lgb = best_lgb_model.predict(X_test)

# Evaluate final model
lgb_accuracy = accuracy_score(y_test, y_pred_lgb)
lgb_f1 = f1_score(y_test, y_pred_lgb, average='weighted')
lgb_precision = precision_score(y_test, y_pred_lgb, average='weighted')
lgb_recall = recall_score(y_test, y_pred_lgb, average='weighted')

print(f"✅ Optimized LightGBM Results:")
print(f"   Accuracy: {lgb_accuracy:.4f}")
print(f"   F1-Score: {lgb_f1:.4f}")
print(f"   Precision: {lgb_precision:.4f}")
print(f"   Recall: {lgb_recall:.4f}")

# Feature importance
print(f"\n📊 Top 15 Most Important Features:")
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(15))

# Show embedding features importance
embedding_features = [col for col in feature_cols if 'emb_' in col]
embedding_importance = feature_importance[feature_importance['feature'].isin(embedding_features)]
print(f"\n📊 Embedding Features Importance (Top 10):")
print(embedding_importance.head(10))

# Show academic features importance
academic_features = [col for col in feature_cols if 'emb_' not in col and 'community' not in col]
academic_importance = feature_importance[feature_importance['feature'].isin(academic_features)]
print(f"\n📊 Academic Features Importance:")
print(academic_importance)


[I 2025-09-28 02:58:48,453] A new study created in memory with name: no-name-db2d43d7-51e6-4386-a490-77ebdba39df7


🚀 LIGHTGBM WITH OPTUNA HYPERPARAMETER TUNING
🔍 Creating Optuna study...
🚀 Running 50 optimization trials...


Best trial: 0. Best value: 0.309626:   2%|▏         | 1/50 [00:06<05:15,  6.44s/it]

[I 2025-09-28 02:58:54,890] Trial 0 finished with value: 0.3096259965694667 and parameters: {'n_estimators': 250, 'learning_rate': 0.2536999076681771, 'max_depth': 10, 'num_leaves': 64, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352, 'min_child_samples': 64}. Best is trial 0 with value: 0.3096259965694667.


Best trial: 0. Best value: 0.309626:   4%|▍         | 2/50 [00:18<07:51,  9.83s/it]

...

[I 2025-09-28 03:05:54,302] Trial 47 finished with value: 0.29734717309398534 and parameters: {'n_estimators': 298, 'learning_rate': 0.1229934688326258, 'max_depth': 9, 'num_leaves': 28, 'subsample': 0.6552492803630611, 'colsample_bytree': 0.7182816667372344, 'reg_alpha': 0.3117067575611571, 'reg_lambda': 0.6478892042133175, 'min_child_samples': 34}. Best is trial 34 with value: 0.31380206837823194.


Best trial: 34. Best value: 0.313802:  98%|█████████▊| 49/50 [07:19<00:11, 11.09s/it]

[I 2025-09-28 03:06:07,559] Trial 48 finished with value: 0.30615546221142514 and parameters: {'n_estimators': 228, 'learning_rate': 0.1136306009836444, 'max_depth': 10, 'num_leaves': 75, 'subsample': 0.8015467078259829, 'colsample_bytree': 0.6832092718580957, 'reg_alpha': 0.056423385332759535, 'reg_lambda': 0.9941341769178521, 'min_child_samples': 20}. Best is trial 34 with value: 0.31380206837823194.


Best trial: 34. Best value: 0.313802: 100%|██████████| 50/50 [07:24<00:00,  8.89s/it]


[I 2025-09-28 03:06:12,957] Trial 49 finished with value: 0.30481197974175267 and parameters: {'n_estimators': 468, 'learning_rate': 0.15888123264714096, 'max_depth': 6, 'num_leaves': 41, 'subsample': 0.7346314912783337, 'colsample_bytree': 0.8167933822916276, 'reg_alpha': 0.55710998676073, 'reg_lambda': 0.78919611985621, 'min_child_samples': 51}. Best is trial 34 with value: 0.31380206837823194.

✅ Optimization Complete!
   Best F1 Score: 0.3138
   Best Parameters:
     n_estimators: 395
     learning_rate: 0.12086552797249225
     max_depth: 9
     num_leaves: 92
     subsample: 0.7650723155381586
     colsample_bytree: 0.800569795502006
     reg_alpha: 0.3230337031045488
     reg_lambda: 0.948240665286624
     min_child_samples: 20

📊 Training Final LightGBM with Best Parameters...
✅ Optimized LightGBM Results:
   Accuracy: 0.3313
   F1-Score: 0.3207
   Precision: 0.3159
   Recall: 0.3313

📊 Top 15 Most Important Features:
                         feature  importance
2   student_ove

In [11]:
# Cell 5: Model Comparison and Selection
print("🏆 MODEL COMPARISON AND SELECTION")
print("=" * 50)

# Compare all models
models_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest', 'LightGBM (Optuna)'],
    'Accuracy': [lr_accuracy, dt_accuracy, rf_accuracy, lgb_accuracy],
    'F1-Score': [lr_f1, dt_f1, rf_f1, lgb_f1],
    'Precision': [lr_precision, dt_precision, rf_precision, lgb_precision],
    'Recall': [lr_recall, dt_recall, rf_recall, lgb_recall]
})

print("📊 Model Performance Comparison:")
print(models_comparison.round(4))

# Find best model based on F1-Score
best_model_idx = models_comparison['F1-Score'].idxmax()
best_model_name = models_comparison.loc[best_model_idx, 'Model']
best_f1 = models_comparison.loc[best_model_idx, 'F1-Score']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   F1-Score: {best_f1:.4f}")
print(f"   Accuracy: {models_comparison.loc[best_model_idx, 'Accuracy']:.4f}")

# Select the best model for saving
if best_model_name == 'Logistic Regression':
    final_model = lr_model
    model_type = 'logistic_regression'
elif best_model_name == 'Decision Tree':
    final_model = dt_model
    model_type = 'decision_tree'
elif best_model_name == 'Random Forest':
    final_model = rf_model
    model_type = 'random_forest'
else:  # LightGBM
    final_model = best_lgb_model
    model_type = 'lightgbm_optimized'

print(f"\n💾 Saving {best_model_name} as the final model...")

# Save the best model
model_package = {
    'model': final_model,
    'model_type': model_type,
    'feature_names': feature_cols,
    'target_name': 'grade_category',
    'class_names': sorted(y_train.unique().tolist()),
    'performance': {
        'accuracy': models_comparison.loc[best_model_idx, 'Accuracy'],
        'f1_score': models_comparison.loc[best_model_idx, 'F1-Score'],
        'precision': models_comparison.loc[best_model_idx, 'Precision'],
        'recall': models_comparison.loc[best_model_idx, 'Recall']
    },
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'total_features': len(feature_cols),
    'optuna_params': study.best_params if model_type == 'lightgbm_optimized' else None
}

joblib.dump(model_package, MODEL_PATH)
print(f"✅ Model saved to: {MODEL_PATH}")

# Detailed classification report for best model
print(f"\n📊 Detailed Classification Report for {best_model_name}:")
print(classification_report(y_test, y_pred_lgb, target_names=label_encoder.classes_))

# Confusion matrix
print(f"\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred_lgb)
print(cm)

# Final summary
print(f"\n🎯 TRAINING COMPLETE!")
print(f"📊 Final Summary:")
print(f"   Best Model: {best_model_name}")
print(f"   F1-Score: {best_f1:.4f}")
print(f"   Training Samples: {len(X_train):,}")
print(f"   Test Samples: {len(X_test):,}")
print(f"   Features: {len(feature_cols)}")
print(f"   Target: Grade Categories (A, B, C, D, F)")
print(f"   Datasets Used: train_processed_comprehensive.csv, test_processed_comprehensive.csv")
print(f"\n🚀 Model ready for deployment!")


🏆 MODEL COMPARISON AND SELECTION
📊 Model Performance Comparison:
                 Model  Accuracy  F1-Score  Precision  Recall
0  Logistic Regression    0.3386    0.3246     0.3262  0.3386
1        Decision Tree    0.3045    0.2956     0.2924  0.3045
2        Random Forest    0.3520    0.3275     0.3183  0.3520
3    LightGBM (Optuna)    0.3313    0.3207     0.3159  0.3313

🏆 Best Model: Random Forest
   F1-Score: 0.3275
   Accuracy: 0.3520

💾 Saving Random Forest as the final model...


AttributeError: 'numpy.ndarray' object has no attribute 'unique'